# Train an Image-Classifier using Federated Learning with Differential Privacy
## A complete End-to-End Real World Example - PySyft, PyGrid
* This tutorial aims at demonstrating a high level of privacy using **Secured Federated Learning** and **Differential Privacy based on the Laplace-Mechanism** as part of the **PATE framework**. To tackle a problem similair to a real-world-problem, yet still using a well explored example (the privacy tools are of main interest in this tutorial) it was chosen to train an Image-Classifier on the **[Cifar-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html)**. In the [original paper on the PATE framework](https://arxiv.org/pdf/1610.05755.pdf), simpler problems such as training a classifier on MNIST and SVHN were tackled.
* Containing 60000 32x32 pixels coloured images of 10 different classes (airplanes, birds, etc.) this should showcase a reasonably similair task to another real-world example such as the training of a classifier for skin-cancer-classification, which heavily relies on sensitive private data. See [Stanford's Skin Cancer Classification with Deep Learning](https://cs.stanford.edu/people/esteva/nature/) for more information on this specific example.
* To be able to give a complete end-to-end example after having performed the example using `PySyft.VirtualWorkers` we'll be using a Public/Private Grid and actual workers using the PyGrid work.  

### Specific Situation - Real-World-Scenario: 
To make the example as real as possible and to best give a quick overview over the PATE framework for DP along with FL, we define  three different parties taking part in this setup. 
  1. **Data Provider:** Owns personal data, in this case some photographers who all made pictures of some objects in real life and sorted them (into the ten classes for this examle). His/her goal is to either *create/train a classifier* which automates the sorting of the images for him/her or to *participate in creating a general image classifier which can recognize objects on images in general*. Possibly he also wants somebody to *host the classifier to speed up inference time*. 
  2. **Model Provider:** Creates the model, in this case the classifier-model. His/her goal is to use his model-architecture and train it on the photographers (Data Provider) personal data and to then make this trained model available to a broad range of user. 
  3. **Hosting Provider:** Provides computing ressources along possible hosting capabilities. His/her goal is to provide the Data and Model Provider with the neccessary computing power for training and hosting capabilities for deployment. 
  * **How is this example representative for the need/usage of privacy preserving ML?**
    This example portrays some of the key characteristics of privacy-preserving ML. 
      * **Combining Knowledge:** A single photographer wouldn't provide enough data to train a good image classifier and wouldn't have the knowledge to build a classifier. Together with different other (possibly even competing photographers) and a model provider he can train a good model together with the other parties. 
      * **Federated Learning:** He doesn't want his sorted photographs to be accessable by either the other competing photographers or the model creator. (He wants to sell his photographs)
      * **Differential Privacy:** Classifiers as ML-Models in general can memorize specific parts of the training data, making it possible to retrieve information about the training data (or even parts of some datapoints). In this case the photographer doesn't want users of the classifier (e.g.: the competing photographers) to be able to retrieve information about his unique way of photograhping (e.g.: key motives, etc.)  
      * All the above points are also exactly the critical points when trainng a classifier e.g. on a skin-cancer dataset.

### The PP-techniques that will be used here:
* **FL** - Federated Learning: We will be using **SMPC-Encrypted Federated Learning** *(For a quick overview see below)*
* **DP** - Differential Privacy: We will be using the **Laplace-Mechanism** as part of the secure aggregation in the PATE procedure *(More information below)*


### The PP-tools that will be used here:
* **PySyft**
  * Plans 
  * Protocolls
  * VirtualWorkers
* **PyGrid** 
  * Gateway
  * GridNode
  
**TODO: CHECK SPECIFIC TYPES AND IN WHICH LIBRARIES THEY ARE! (Apparently currently changing)**

Author:
- Nicolas Remerscheid - GitHub: [@NiWaRe](https://github.com/NiWaRe)

References: 
- References of different authors 
  *This example/usecase is heavily based in previous tutorials on PySyft and PyGrid from ......*
 
***TODO: LINK SPECIFIC LEARNING MATERIALS AS PREREQUESITS AND SPECIFIC AUTHORS.***

# The PATE Framework
## The assumptions 
Introduced in [this paper](https://arxiv.org/pdf/1610.05755.pdf) the *Private Aggregation of Teacher
Ensembles* (PATE) framework consists of the following basic setup. 
It is important to note that the PATE framework assumes a certain situation: 
* **Discrete Model Output:** The PATE framework (acoording to the original paper) assumes that a model should be trained that has a discrete output i.e. it categorizes or classifies some input into categories. In this case this is an image-classifier. 
* **Data structure:** The PATE framework assumes that the following data exists and is stored in the following way:
  * **Private Data:** The private data is stored on multiple workers (photographers here) The private data is *labeled* and each worker has *different datapoints* (different photographs). Furthermore the private data isn't sufficient to train a good classifier alone.  
  * **Public Data:** In addition to the private data PATE assumes that there exists a second larger dataset of *unlabeled, public and unsensitive* images. In this case for example this could be very large database of pictures from various newspapers. They aren't labeled and everybody can access them since the newspapers decided to share them, as they don't sell them directly. 
* **Semi-Supervised Learning:** Given the particular datasets that are given PATE assumes that we want to leverage smaller labeled datasets (normally used for supervised learning) together with an unlabeled dataset (normally used for unsupervised learning) to train a final model, meaning we want to train it using semi-supervised learning. 


## The procedure
(For more info see in the learning material section above, notably Andrew Trask's tutorial) <br>
This image shows an overview over the procedure: 
<img src="./material/PATE_framework_overview_from_paper.png" width="600">

1. STEP: **Multiple Teacher-Models are trained on each of the private datasets**
  * This is also where we want to use **Federated Learning** to not *directly* access the data at all. To make sure that the photographers don't see the model of the model provider (he/she sells that) we'll be encrypting both the model and the data via Additive Secret Sharing using Secure Multi-Party Computing (SMPC). *Overview on that later.*
2. STEP: **For each unlabeled datapoint in the public dataset teacher-models jointly predict a label**
  * This is where noise is added for **Differential Privacy**. 
3. STEP: **One Student-Model is trained on the newly labeled public dataset**
  * This is where **the final model** is created. 

# Setting Up and Testing without Grid
First we want to implement the procedure in raw form and as an automated protocoll and test it with a simple infrastructure using `PySyft.VirtualWorkers()` to then later deploy it to an actual grid-infrastructure whith actual workers. 

## Before we start: Setting up the real-case scenario
We will beginn with VirtualWorkers and no grid to test the different other components and protocolls (FL, PATE). <br>
First we will create all the workers with correct data stored on them. This step isn't necessary in a normal scenario, as a group of workers with data already exist. 

In [1]:
# TODO: Beginning with standard setup , Setup workers, etc. 
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import numpy
import syft as sy 

hook = sy.TorchHook(torch)

In [2]:
# Note: 
# - Federated Dataset could be used 
# - data attribute with VirtualWorkes can't be used to not use efficient data loading of DataLoader
#   everything would be stored in one place....think further about that (relook at forum)
# - DOC. better

# Create the VirtualWorkers 
# TODO: Potentially extend the function to support creation of actual workers with additional argument 
def createVWorkers(w_number, w_name):
    worker = []
    for i in range(w_number):
        worker.append(sy.VirtualWorker(hook, id=w_name+str(i)))
    return worker

# Takes in worker array and a dataset which should be stored equally on workers 
def PATE_DATA_VSETUP(teachers, student, dataset, batch_size, ratio=0.4):
    # No differentiation between training and testing is made, 
    # because in reality the data providers won't have the data seperated for ML 
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    numb_workers = len(teachers)+1
    increment = len(dataset) / numb_workers

    dist_dataset_teachers = {"data":[], "target":[]}
    dist_dataset_student = {"data":[]}
    
    for i, (data, target) in enumerate(dataloader):
        ### PySyft ###
        # Teachers: The first 40% (normally less than on public) of the data
        if i < int(ratio * (len(dataset)/batch_size)):
            data = data.tag("cifar").send(teachers[i%len(teachers)])
            target = target.tag("cifar").send(teachers[i%len(teachers)])
            # Store the pointers to the send data.
            dist_dataset_teachers["data"].append(data)
            dist_dataset_teachers["target"].append(target)
        # Student: The larger part is unlabeled 
        else: 
            data = data.tag("cifar").send(student)
            # Store the pointers to the send data.
            dist_dataset_student["data"].append(data)
        
    return dist_dataset_teachers, dist_dataset_student 

# Normalize data and convert to torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
# Get Cifar10 Dataset from torchvision.datasets
cifar10_train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)

# Don't use to store on workers because we want to test performance of decentralized training at the end
cifar10_test_data = datasets.CIFAR10('data', train=False,
                              download=True, transform=transform)

# Args
batch_size = 20 
ratio = 0.4

# Private Data Provider
teachers = createVWorkers(3, "photographer")

# Public Data Provider
student = createVWorkers(1, "public_cloud")

# Distribute Data 
dist_dataset_teachers, dist_dataset_student = PATE_DATA_VSETUP(teachers, student,
                                                                 cifar10_train_data, batch_size)

# Model Provider
model_provider = createVWorkers(1, "model_provider")

# Hosting Provider 
hosting_provider = createVWorkers(3, "hosting_provider")

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# As an example: Get the target of the first example in the first batch -> Note: Not one-hot-encoded
print(dist_dataset_teachers["target"][0][0])
print(dist_dataset_student["data"][0][0])


(Wrapper)>[PointerTensor | me:42183060114 -> photographer0:85629921556]
(Wrapper)>[PointerTensor | me:66639053926 -> public_cloud0:88462510572]


## Creating Model + Rest Setup

In [ ]:
### Think about Validation (from test or training data)

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

## PATE Step 1: Traning Teacher Models 
Here FL learning will be done. 


In [ ]:
# TODO: Implement SMPC-FL to train teacher models 
#       (Possibly first test in raw form, then implemented as Plan/Part of Protocoll)

In [ ]:
# TODO: for protocolls check again how the pate-framework is already implemented in PySyft

## PATE Step 2: Label Public Dataset with aggregated teacher-predictions
Here DP aggregation will be done.

In [ ]:
# TODO: Implement DP-secure aggregation of teacher predictions on public dataset 

## PATE Step 3: Train Student Model
Here Student Model will be trained 

In [ ]:
# TODO: Implement training of student model 

## Testing of PATE Procedure 
Here using the testing infrastructure the PATE procedure will be tested. <br>
The goal is to implement the PATE procedure as a `PySyft.protocoll` using `PySyft.plans` which also should be tested as a general automated procedure before deploying everything on actual workers and a grid. 

In [ ]:
# TODO: Do testing of complete procedure

# The final project on the grid
Here the final real end-to-end example of photographers training a image-classfier together with a model provider should be showcased using an actual grid, whith a gateway and workers which could actually run on remote machines. 

In [ ]:
# TODO: Setting up a grid with workers + documentation + further links to grid tutorials 

# Performance Evaluation 
* **Performance Evaluation:** *Depending on time can be extended or not*
  Analyse Performance of Decentralized Training with normal Training, Possible metrics: 
    * Training Time: Convergence Speed, Compuation Time (incl. communication, etc.)
    * Model-Performance: Relevant Metrics (Accuracy, etc.)
    * Privacy-Leakage-Anlysis in different security situations (active and passive security)
    * Model-Robustness: Impact of decentralized 
    * Model-Fairness: Impact of decentralized 